In [0]:
spark.conf.set(
    "fs.azure.account.key.goodreadsreviews60106541.dfs.core.windows.net",
    "<AccountKey>"
)


In [0]:
# Load the books dataset from the silver layer
books = spark.read.parquet(
"abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/processed/books/"
)
# Load the authors dataset from the silver layer
authors = spark.read.parquet(
"abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/processed/authors/"
)
# Display the first few records to confirm the data was loaded correctly
books.show(5)
authors.show(5)
# Display the columns and their data types to verify the schema
books.printSchema()
authors.printSchema()


+----------+------------------+------------+-------------+----------+--------+--------------+-----------+--------------------+---------+--------------------+--------------------+---------+---------------+-------------+-----------------+-------------------+----------------+--------------------+--------------------+-------+-------------+-------+--------------------+--------------------+
|      isbn|text_reviews_count|country_code|language_code|      asin|is_ebook|average_rating|kindle_asin|         description|   format|                link|           publisher|num_pages|publication_day|       isbn13|publication_month|edition_information|publication_year|                 url|           image_url|book_id|ratings_count|work_id|               title|title_without_series|
+----------+------------------+------------+-------------+----------+--------+--------------+-----------+--------------------+---------+--------------------+--------------------+---------+---------------+-------------+------

In [0]:

from pyspark.sql.functions import col, length, trim, count, when
# Since reviews worked with me 
reviews = spark.read.parquet(
"abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/processed/reviews/"
)

reviews.show(5, truncate=False)
reviews.printSchema()
# Basic profiling: counts and potential issues
total_rows = reviews.count()
null_review_id = reviews.filter(col("review_id").isNull()).count()
null_book_id = reviews.filter(col("book_id").isNull()).count()
null_user_id = reviews.filter(col("user_id").isNull()).count()
null_rating = reviews.filter(col("rating").isNull()).count()
empty_text = reviews.filter( (col("review_text").isNull()) | (trim(col("review_text")) ==
"") ).count()
print(f"Total rows: {total_rows}")
print(f"NULL review_id: {null_review_id}, NULL book_id: {null_book_id}, NULL user_id:{null_user_id}, NULL rating: {null_rating}")
print(f"Empty/NULL review_text: {empty_text}")

+--------------------------------+--------+--------------------------------+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
from pyspark.sql.functions import col, trim, length
# Start from the existing Parquet-loaded DataFrame
# (Assumes you already did: reviews = spark.read.parquet(".../processed/reviews/"))
df = reviews
# 1) Drop rows missing critical keys
df = df.filter(
col("review_id").isNotNull() &
col("book_id").isNotNull() &
col("user_id").isNotNull()
)
# 2) Enforce rating to be integer in [1..5]
df = df.withColumn("rating_int", col("rating").cast("int"))
df = df.filter(
col("rating_int").isNotNull() &
(col("rating_int") >= 1) &
(col("rating_int") <= 5)
)
# 3) Normalize text; drop empty or ultra-short reviews (<10 chars after trim)
df = df.withColumn("review_text", trim(col("review_text")))
df = df.filter(
col("review_text").isNotNull() &
(length(col("review_text")) >= 10)
)
# 4) De-duplicate by review_id (keep arbitrary first; refine if you have timestamps)
df = df.dropDuplicates(["review_id"])
# 5) Select final shape
reviews_clean = df.select(
"review_id",
"book_id",
"user_id",
col("rating_int").alias("rating"),
"review_text"
)

In [0]:
# Write the cleaned reviews back to the silver layer (overwrite)
reviews_clean.write.mode("overwrite").parquet(
"abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/processed/reviews/"
)
# Sanity check: re-read from disk and inspect schema and a few rows
reviews_verified = spark.read.parquet(
"abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/processed/reviews/"
)
reviews_verified.printSchema()
reviews_verified.show(5, truncate=False)
print(f"Written cleaned rows: {reviews_verified.count()}")

root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)

+--------------------------------+--------+--------------------------------+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

**HomeWork Part 1**

In [0]:
silver_base = "abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/processed"
books_path = f"{silver_base}/books/"
authors_path = f"{silver_base}/authors/"
reviews_path = f"{silver_base}/reviews/"
book_authors_path = f"{silver_base}/book_authors/"   
book_authors_rejects_path = f"{silver_base}/book_authors_rejects/"


In [0]:
books = spark.read.parquet(books_path)
authors = spark.read.parquet(authors_path)
reviews_verified = spark.read.parquet(reviews_path)   

books.printSchema()
books.show(5, truncate=False)

authors.printSchema()
authors.show(5, truncate=False)

reviews_verified.printSchema()
reviews_verified.show(5, truncate=False)


root
 |-- isbn: string (nullable = true)
 |-- text_reviews_count: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- is_ebook: string (nullable = true)
 |-- average_rating: string (nullable = true)
 |-- kindle_asin: string (nullable = true)
 |-- description: string (nullable = true)
 |-- format: string (nullable = true)
 |-- link: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- num_pages: string (nullable = true)
 |-- publication_day: string (nullable = true)
 |-- isbn13: string (nullable = true)
 |-- publication_month: string (nullable = true)
 |-- edition_information: string (nullable = true)
 |-- publication_year: string (nullable = true)
 |-- url: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- ratings_count: string (nullable = true)
 |-- work_id: string (nullable = true)
 |-- title: string (null

In [0]:
books.select("book_id").show(5)
print("Columns:", books.columns)


+-------+
|book_id|
+-------+
|5333265|
|1333909|
|7327624|
|6066819|
| 287140|
+-------+
only showing top 5 rows
Columns: ['isbn', 'text_reviews_count', 'country_code', 'language_code', 'asin', 'is_ebook', 'average_rating', 'kindle_asin', 'description', 'format', 'link', 'publisher', 'num_pages', 'publication_day', 'isbn13', 'publication_month', 'edition_information', 'publication_year', 'url', 'image_url', 'book_id', 'ratings_count', 'work_id', 'title', 'title_without_series']


In [0]:
# Join reviews with books on book_id
rev_books = reviews_verified.join(books, on="book_id", how="left")

print("Joined reviews + books:")
rev_books.select("book_id", "title", "review_id", "user_id", "rating").show(5, truncate=False)


Joined reviews + books:
+--------+----------------------------------------------+--------------------------------+--------------------------------+------+
|book_id |title                                         |review_id                       |user_id                         |rating|
+--------+----------------------------------------------+--------------------------------+--------------------------------+------+
|21536151|Easy Virtue (Virtue, #1)                      |f1841fd770ab345f2c9a8ec73f005f82|01ec1a320ffded6b2dd47833f2c8e4fb|5     |
|8667848 |A Discovery of Witches (All Souls Trilogy, #1)|a71566f5c6a2d61a5e0a68353156052b|b6298225d447a35c7c225a288757793e|3     |
|1457464 |Ana's Story: A Journey of Hope                |1f1ce0da8b8e7e18c937b2d83812cbd1|7b77c5caf0e0df193360511ac7f7288c|2     |
|14743784|Unbreakable (Unraveling, #2)                  |fb171944dde5179fe4a4e2877920b16e|012515e5802b2e0f42915118c90fa04b|4     |
|11505797|Beautiful Disaster (Beautiful, #1)            |c8

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

# Create a bridge by assigning each book an author in a demo way
# (In the real dataset, this mapping would come from the original CSV source)
book_authors = books.select("book_id").withColumn("author_id", (monotonically_increasing_id() % authors.count()))

book_authors.show(5)
print("Bridge table created with artificial mapping:")
book_authors.printSchema()



+-------+---------+
|book_id|author_id|
+-------+---------+
|5333265|        0|
|1333909|        1|
|7327624|        2|
|6066819|        3|
| 287140|        4|
+-------+---------+
only showing top 5 rows
Bridge table created with artificial mapping:
root
 |-- book_id: string (nullable = true)
 |-- author_id: long (nullable = true)



In [0]:
# === Curate GOLD (deterministic author mapping + extra columns) ===
from pyspark.sql import functions as F, Window

base      = "abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net"
gold_out  = f"{base}/gold/curated_reviews_delta/"   # final Delta table for Fabric

# 1) Select/rename needed book fields and build a date
books_sel = (
    books.select(
        "book_id",
        "title",
        F.col("language_code").alias("language"),
        F.col("ratings_count").alias("n_votes"),
        F.make_date(
            F.col("publication_year").cast("int"),
            F.coalesce(F.col("publication_month").cast("int"), F.lit(1)),
            F.coalesce(F.col("publication_day").cast("int"), F.lit(1))
        ).alias("date_added")
    )
)

# 2) Deterministic book→author bridge (no original bridge present)
#    Create a stable index over authors, then map each book by hash(book_id) % author_cnt
authors_idx = (
    authors.select("author_id").distinct()
           .withColumn("idx", F.row_number().over(Window.orderBy(F.col("author_id").cast("long"))) - 1)
)
author_cnt = authors_idx.count()

book_idx = (
    books.select("book_id")
         .withColumn("idx", (F.abs(F.hash("book_id")) % F.lit(author_cnt)).cast("long"))
)

book_authors = (
    book_idx.join(authors_idx, "idx", "left")
            .select("book_id", "author_id")
)

# 3) Join reviews + books + (bridged) authors → select final 11 columns
cur = (
    reviews_verified.alias("r")
        .join(books_sel.alias("b"), "book_id", "left")
        .join(book_authors.alias("ba"), "book_id", "left")
        .join(authors.select("author_id","name").alias("a"), "author_id", "left")
        .select(
            "review_id", "book_id", "title",
            "author_id", "name", "user_id",
            "rating", "review_text",
            "language", "n_votes", "date_added"
        )
)

# 4) Types + write as Delta (overwrite schema)
cur = cur.withColumn("n_votes", F.col("n_votes").cast("int"))

cur.write.format("delta") \
   .mode("overwrite") \
   .option("overwriteSchema", "true") \
   .save(gold_out)

# 5) Quick verification
df = spark.read.format("delta").load(gold_out)
df.printSchema()
df.select("book_id","author_id","name","language","n_votes","date_added").show(10, truncate=False)


root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language: string (nullable = true)
 |-- n_votes: integer (nullable = true)
 |-- date_added: date (nullable = true)

+--------+---------+--------------------+--------+-------+----------+
|book_id |author_id|name                |language|n_votes|date_added|
+--------+---------+--------------------+--------+-------+----------+
|18089323|4094043  |Tracy Woelfel       |        |45     |NULL      |
|28372202|3093262  |Jo-Anne Vandermeulen|ind     |51     |2016-01-21|
|7937843 |4124792  |Dominika Stec       |eng     |522469 |2010-09-13|
|140082  |4332619  |Marcus Birro        |en-GB   |182560 |2006-05-06|
|30739547|7072825  |Herta Balling       |en-US   |5806   |2016-10-25|

In [0]:
# write to the Delta path Fabric reads
cur.write.format("delta") \
   .mode("overwrite") \
   .option("overwriteSchema","true") \
   .save("abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/gold/curated_reviews_delta/")


In [0]:
from pyspark.sql import functions as F
cur = (cur
  .withColumn("language", F.when(F.col("language").isin("", "en-GB","en_US"), "eng").otherwise(F.col("language")))
)
